# Imports & Connections

In [1]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np
from datetime import datetime
from sqlalchemy import create_engine

In [2]:
DB = {
    'servername': 'DESKTOP-IT4OHPV\SQLEXPRESS',
    'database': 'johari_Herkansing_NEW1'
}

ssms_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                     ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')

# Connection String (update with your details)
connection_string = "mssql+pyodbc://@DESKTOP-IT4OHPV\SQLEXPRESS/johari_Herkansing_NEW1?driver=SQL+Server+Native+Client+11.0?trusted_connection=yes"
ssms_cursor = ssms_conn.cursor()   

# Create Engine
engine = create_engine(connection_string)

<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:10: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:10: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Noah\AppData\Local\Temp\ipykernel_63792\3369659595.py:2: SyntaxWarning: invalid escape sequence '\S'
  'servername': 'DESKTOP-IT4OHPV\SQLEXPRESS',
C:\Users\Noah\AppData\Local\Temp\ipykernel_63792\3369659595.py:10: SyntaxWarning: invalid escape sequence '\S'
  connection_string = "mssql+pyodbc://@DESKTOP-IT4OHPV\SQLEXPRESS/johari_Herkansing_NEW1?driver=SQL+Server+Native+Client+11.0?trusted_connection=yes"


In [3]:
# Local Connections
sqlite_conn = sqlite3.connect('../data/processed/dwh.sqlite')
go_sales_conn = sqlite3.connect('../data/raw/go_sales.sqlite')
go_staff_conn = sqlite3.connect('../data/raw/go_staff.sqlite')
go_crm_conn = sqlite3.connect('../data/raw/go_crm.sqlite')

# Fact tables
fact_tables = ['training', 'satisfaction', 'returned_item', 'orders', 'sales_targetData', 'sales_product_forecast', 'sales_inventory_levels']
fact_dfs = {table: pd.read_sql(f'SELECT * FROM {table}', ssms_conn) for table in fact_tables}

# Dimension tables
dim_tables = ['product', 'sales_staff', 'course', 'satisfaction_type', 'return_reason', 'order_method', 'retailer_contact']
dim_dfs = {table: pd.read_sql(f'SELECT * FROM {table}', ssms_conn) for table in dim_tables}

# Test Output
orders_df = fact_dfs['orders']
product_df = dim_dfs['product']
#product_df.head()

C:\Users\Noah\AppData\Local\Temp\ipykernel_63792\1395309261.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fact_dfs = {table: pd.read_sql(f'SELECT * FROM {table}', ssms_conn) for table in fact_tables}
C:\Users\Noah\AppData\Local\Temp\ipykernel_63792\1395309261.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_dfs = {table: pd.read_sql(f'SELECT * FROM {table}', ssms_conn) for table in dim_tables}


# Surrogate Key Creation

In [ ]:
# Function to add surrogate key to dimension tables
def add_surrogate_key(conn, table_name):
    cursor = conn.cursor()
    # Add SK column
    cursor.execute(f"ALTER TABLE {table_name} ADD SK INT IDENTITY(1,1);")
    # Add EffectiveDate column
    cursor.execute(f"ALTER TABLE {table_name} ADD EffectiveDate DATETIME NOT NULL DEFAULT '1900-01-01';")
    # Add EndDate column
    cursor.execute(f"ALTER TABLE {table_name} ADD EndDate DATETIME NULL;")
    # Add IsActive column
    cursor.execute(f"ALTER TABLE {table_name} ADD IsActive BIT NOT NULL DEFAULT 1;")
    conn.commit()

# Add surrogate keys to dimension tables
for table in dim_tables:
    add_surrogate_key(ssms_conn, table)

# Foreign Surrogate Key Creation

In [ ]:
# Function to add surrogate foreign key columns to fact tables
def add_surrogate_foreign_key_columns(conn, fact_table, key_mappings):
    cursor = conn.cursor()
    for _, (fact_table, foreign_key_column) in key_mappings.items():
        cursor.execute(f"ALTER TABLE {fact_table} ADD {foreign_key_column} INT;")
    conn.commit()

# Add surrogate foreign key columns to fact tables
for fact_table in fact_tables:
    if fact_table == 'orders':
        key_mappings = {
            #Orders
            'PRODUCT_id': ('orders', 'PRODUCT_SFK'),
            'ORDER_METHOD_id': ('orders', 'ORDER_METHOD_SFK'),
            'RETAILER_CONTACT_id': ('orders', 'RETAILER_CONTACT_SFK'),
            'RETURNED_REASON': ('orders', 'RETURNED_REASON_SFK'),

            #Sales_targetData
            'PRODUCT_id': ('sales_targetData', 'PRODUCT_SFK'),
            'SALES_STAFF_id': ('sales_targetData', 'SALES_STAFF_SFK'),

            #Sales_product_forecast
            'PRODUCT_id': ('sales_product_forecast', 'PRODUCT_SFK'),

            #Sales_inventory_levels
            'PRODUCT_id': ('sales_inventory_levels', 'PRODUCT_SFK'),

            #Satisfaction
            'SATISFACTION_TYPE_id': ('satisfaction', 'SATISFACTION_TYPE_SFK'),
            'SALES_STAFF_id': ('satisfaction', 'SALES_STAFF_SFK'),

            #Training
            'COURSE_id': ('training', 'COURSE_SFK'),
            'SALES_STAFF_id': ('training', 'SALES_STAFF_SFK'),

            #Returned_item
            'RETURN_REASON_id': ('returned_item', 'RETURN_REASON_SFK'),
        }
        #add_surrogate_foreign_key_columns(ssms_conn, fact_table, key_mappings)

def insert_sfk_sales_targetData(conn):
    cursor = conn.cursor()
    sql = """
    UPDATE sales_targetData
    SET product_SFK = b.SK
    FROM sales_targetData a
    LEFT JOIN PRODUCT b
        ON b.id = a.product_id
        AND datefromparts(a.year, a.period, 1) BETWEEN b.EffectiveDate AND ISNULL(b.EndDate, '20991231')
    """
    try:
        cursor.execute(sql)
        conn.commit()
        print("Successfully inserted/updated SFKs in sales_targetData.")
    except Exception as e:
        conn.rollback()
        print(f"Error inserting/updating SFKs in sales_targetData: {e}")
    #finally:
        #cursor.close()


def populate_foreign_keys_Sales_targetData(conn, fact_table, sfk_column, dim_table, foreign_key_column, product_id):
    """
    Updates the surrogate foreign key in a fact table based on a dimension table with temporal logic.

    Parameters:
    - conn: Database connection object.
    - fact_table: Name of the fact table.
    - sfk_column: Name of the surrogate foreign key column in the fact table.
    - dim_table: Name of the dimension table.
    - foreign_key_column: Name of the foreign key column in the fact table that matches the natural key in the dimension table.
    - product_id: The product ID to filter on in the WHERE clause.
    """
    cursor = conn.cursor()
    sql = f"""
    UPDATE {fact_table}
    SET {sfk_column} = (
        SELECT {dim_table}.SK
        FROM {dim_table}
        JOIN {fact_table} ON {dim_table}.id = {fact_table}.{foreign_key_column}
        WHERE datefromparts({fact_table}.year, {fact_table}.period, 1) BETWEEN {dim_table}.EffectiveDate AND ISNULL({dim_table}.EndDate, '20991231')
        AND {fact_table}.product_id = {product_id}
    )
    WHERE EXISTS (
        SELECT 1
        FROM {dim_table}
        JOIN {fact_table} ON {dim_table}.id = {fact_table}.{foreign_key_column}
        WHERE datefromparts({fact_table}.year, {fact_table}.period, 1) BETWEEN {dim_table}.EffectiveDate AND ISNULL({dim_table}.EndDate, '20991231')
        AND {fact_table}.product_id = {product_id}
    )
    """
    try:
        cursor.execute(sql)
        conn.commit()
        print(f"Successfully updated SFKs in {fact_table}.")
    except Exception as e:
        conn.rollback()
        print(f"Error updating SFKs in {fact_table}: {e}")
    finally:
        cursor.close()

# Populate Foreign Surrogate Keys

In [ ]:
# Populate SFKs for the 'orders' fact table
#populate_foreign_keys(ssms_conn, 'orders', 'product', 'id', 'product_id', 'PRODUCT_SFK')
#populate_foreign_keys(ssms_conn, 'orders', 'order_method', 'order_method_id', 'order_method_code', 'ORDER_METHOD_SFK')
#populate_foreign_keys(ssms_conn, 'orders', 'RETAILER_CONTACT', 'retailer_id', 'retailer_contact_code', 'RETAILER_CONTACT_SFK')
#populate_foreign_keys(ssms_conn, 'orders', 'RETURN_REASON', 'return_reason_id', '', 'RETURNED_REASON_SFK')

# Populate SFKs for the 'sales_targetData' fact table
#insert_sfk_sales_targetData(ssms_conn)

# Populate SFKs for the 'sales_product_forecast' fact table
#populate_foreign_keys(ssms_conn, 'sales_product_forecast', 'product', 'id', 'product_id', 'PRODUCT_SFK')

# Populate SFKs for the 'sales_inventory_levels' fact table
#populate_foreign_keys(ssms_conn, 'sales_inventory_levels', 'product', 'id', 'product_id', 'PRODUCT_SFK')

# Populate SFKs for the 'satisfaction' fact table
#populate_foreign_keys(ssms_conn, 'satisfaction', 'satisfaction_type', 'id', 'satisfaction_type_id', 'SATISFACTION_TYPE_SFK')
#populate_foreign_keys(ssms_conn, 'satisfaction', 'sales_staff', 'sales_staff_id', 'staff_id', 'SALES_STAFF_SFK')

# Populate SFKs for the 'training' fact table
#populate_foreign_keys(ssms_conn, 'training', 'course', 'id', 'course_id', 'COURSE_SFK')
#populate_foreign_keys(ssms_conn, 'training', 'sales_staff', 'sales_staff_id', 'staff_id', 'SALES_STAFF_SFK')

# Populate SFKs for the 'returned_item' fact table
#populate_foreign_keys(ssms_conn, 'returned_item', 'return_reason', 'id', 'return_reason_id', 'RETURN_REASON_SFK')


In [4]:
# Test Output
orders_df = fact_dfs['orders']
product_df = dim_dfs['product']
sales_targetData_df = fact_dfs['sales_targetData']
sales_targetData_df.head()

,id,staff_id,staff_position_en,year,period,target,product_id,product_name,product_type,product_line,retailer_id,retailer_name,retailer_type_en,PRODUCT_SFK,SALES_STAFF_SFK
0,1,6,Level 1 Sales Representative,2021,5,489,53,Firefly Climbing Lamp,Climbing Accessories,Mountaineering Equipment,6,Altitudes extrêmes,Outdoors Shop,53,None
1,2,6,Level 1 Sales Representative,2021,5,666,55,Firefly Rechargeable Battery,Climbing Accessories,Mountaineering Equipment,6,Altitudes extrêmes,Outdoors Shop,55,None
2,3,6,Level 1 Sales Representative,2021,5,682,56,Granite Chalk Bag,Climbing Accessories,Mountaineering Equipment,6,Altitudes extrêmes,Outdoors Shop,56,None
3,4,6,Level 1 Sales Representative,2021,5,3193,76,Bear Edge,Knives,Personal Accessories,6,Camping Sauvage,Outdoors Shop,76,None
4,5,6,Level 1 Sales Representative,2021,5,7236,85,Glacier GPS Extreme,Navigation,Personal Accessories,6,Camping Sauvage,Outdoors Shop,85,None


In [ ]:
product_df.head()

# Simulate Slowly Changing Dimensions Type 2

In [8]:
def simulate_scd_type_2_with_fact_update():
    # Step 1: Select a random active row from the PRODUCT table
    select_query = "SELECT TOP 1 * FROM dbo.PRODUCT WHERE IsActive = 1 ORDER BY NEWID()"
    ssms_cursor.execute(select_query)
    row = ssms_cursor.fetchone()
    if row is None:
        print("No active rows found.")
        return

    # Retrieve the maximum id value from the PRODUCT table
    id_query = "SELECT MAX(id) FROM dbo.PRODUCT"
    ssms_cursor.execute(id_query)
    max_id = ssms_cursor.fetchone()[0]
    new_id = max_id + 1 if max_id is not None else 1
    
    # Step 2: Update the previously active row to set IsActive to 0 and update EndDate
    effective_date = datetime.now()
    update_query = "UPDATE dbo.PRODUCT SET IsActive = 0, EndDate = ? WHERE SK = ?"
    ssms_cursor.execute(update_query, (effective_date, row[-4]))
    ssms_conn.commit()

    # Assuming `row` contains the selected row from the database and `id` is at index 0
    old_id = row[0]  # Use the existing id directly

    # Step 3: Insert a new row with the same id (unusual for SCD Type 2)
    params = (old_id,) + row[1:-4] + (effective_date, None, 1)
    insert_query = """
    INSERT INTO dbo.PRODUCT 
    (id, name, image, description, type_id, type_name, line_id, line_name, introduction_date, production_cost, margin, language, EffectiveDate, EndDate, IsActive) 
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    ssms_cursor.execute(insert_query, params)
    ssms_conn.commit()

    # Fetch the last inserted SK
    ssms_cursor.execute("SELECT SCOPE_IDENTITY() AS NewSK")
    new_sk = ssms_cursor.fetchone()[0]

    # Optional Step 4: Update the sales_targetData table to reflect the new SK for the product
    update_sales_query = "UPDATE sales_targetData SET product_SFK = ? WHERE product_id = ? AND product_SFK = ?"
    ssms_cursor.execute(update_sales_query, (new_sk, row[1], row[0]))  # Adjust indices based on your table structure
    ssms_conn.commit()

# Assuming ssms_cursor and ssms_conn are already defined and connected to your database
simulate_scd_type_2_with_fact_update()

# Close the cursor and connection
#ssms_cursor.close()
#ssms_conn.close()

DataError: ('22018', "[22018] [Microsoft][ODBC SQL Server Driver][SQL Server]Conversion failed when converting the nvarchar value 'Granite Climbing Helmet' to data type int. (245) (SQLExecDirectW)")

In [ ]:
def simulate_scd_type_2():
    # Step 1: Select a random active row from the PRODUCT table
    select_query = "SELECT TOP 1 * FROM dbo.PRODUCT WHERE IsActive = 1 ORDER BY NEWID()"
    ssms_cursor.execute(select_query)
    row = ssms_cursor.fetchone()
    if row is None:
        print("No active rows found.")
        return

    # Retrieve the maximum id value from the PRODUCT table
    id_query = "SELECT MAX(id) FROM dbo.PRODUCT"
    ssms_cursor.execute(id_query)
    max_id = ssms_cursor.fetchone()[0]
    new_id = max_id + 1 if max_id is not None else 1
    
    # Step 2: Update the previously active row to set IsActive to 0 and update EndDate
    effective_date = datetime.now()
    update_query = "UPDATE dbo.PRODUCT SET IsActive = 0, EndDate = ? WHERE SK = ?"
    ssms_cursor.execute(update_query, (effective_date, row[-4]))
    ssms_conn.commit()

    # Step 3: Insert a new row without specifying the SK (identity column)
    params = (new_id,) + row[1:-4] + (effective_date, None, 1)
    insert_query = "INSERT INTO dbo.PRODUCT (id, name, image, description, type_id, type_name, line_id, line_name, introduction_date, production_cost, margin, language, EffectiveDate, EndDate, IsActive) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    ssms_cursor.execute(insert_query, params)
    ssms_conn.commit()

# Assuming ssms_cursor and ssms_conn are already defined and connected to your database
simulate_scd_type_2()

# Close the cursor and connection
#ssms_cursor.close()
#ssms_conn.close()

In [ ]:
cursor = ssms_conn.cursor()

# Step 1: Update the old record
update_sql = """
UPDATE product
SET IsActive = 0, EndDate = ?
WHERE id = 1 AND IsActive = 1
"""
cursor.execute(update_sql, datetime.now())

# Step 2: Insert the new record
insert_sql = """
INSERT INTO product (id, name, IsActive, EffectiveDate, EndDate)
VALUES (?, ?, ?, ?, ?, ?)
"""

random_image_value = "sdfdsfdsfdsff"
# New ID should be generated appropriately, here just for demonstration
new_id = 2  # This should be dynamically generated or omitted if auto-increment
cursor.execute(insert_sql, new_id, 'TrailChef Water Teepee', 1, datetime.now(), None, random_image_value)

# Commit the transaction
ssms_conn.commit()

# Close the cursor and connection if no longer needed
cursor.close()
ssms_conn.close()

In [ ]:
# Close connections
ssms_conn.close()
sqlite_conn.close()
go_sales_conn.close()
go_staff_conn.close()
go_crm_conn.close()